# Differentially Private Deep Learning: How to use the PyTorch Opacus Library

This script is my implementation of the following [tutorial](https://blog.openmined.org/differentially-private-deep-learning-using-opacus-in-20-lines-of-code/).


Relevant papers include: 

- https://arxiv.org/abs/2206.07758 - demonstrate that you can reverse engineer a neural network and see the initial images

 ### Step 1: Import libraries

In [1]:
import torch
from torchvision import datasets, transforms

In [2]:
import numpy as np
from opacus import PrivacyEngine
from tqdm import tqdm

### Step 2: Load MNIST Data

In [3]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist',
			   train=True, download=True,
               transform=transforms.Compose([transforms.ToTensor(),
               transforms.Normalize((0.1307,), (0.3081,)),]),),
               batch_size=64, shuffle=True, num_workers=1,
               pin_memory=True)

In [4]:
test_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist',
			  train=False,
              transform=transforms.Compose([transforms.ToTensor(),
              transforms.Normalize((0.1307,), (0.3081,)),]),),
              batch_size=1024, shuffle=True, num_workers=1,
              pin_memory=True)

### Step 3: Creating a PyTorch Neural Network Classification Model and Optimizer

In [9]:
model = torch.nn.Sequential(torch.nn.Conv2d(1, 16, 8, 2, padding=3),
							torch.nn.ReLU(),
                            torch.nn.MaxPool2d(2, 1), 
                            torch.nn.Conv2d(16, 32, 4, 2), 
                            torch.nn.ReLU(), 
                            torch.nn.MaxPool2d(2, 1), 
                            torch.nn.Flatten(), 
                            torch.nn.Linear(32 * 4 * 4, 32), 
                            torch.nn.ReLU(), 
                            torch.nn.Linear(32, 10))

optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

### Step 4: Attaching a Differential Privacy Engine to the Optimizer

In [13]:
privacy_engine = PrivacyEngine(model, 
							   batch_size=64, 
                               sample_size=60000,  
                               alphas=range(2,32), 
                               noise_multiplier=1.3, 
                               max_grad_norm=1.0,)

privacy_engine.attach(optimizer)

TypeError: PrivacyEngine.__init__() got an unexpected keyword argument 'batch_size'